In [1]:
# What is the question?
# Are the major stocks the best investments?
# What are the best to buy for next year (more current data but a ton of stocks)
# What are the safest types of stocks to buy by stock type (historical data but only a few specific stocks)

# Predicting what stocks would do well

# function that adds new entries to the database
# based on new entries, machine learning model is further trained
# tells us what stocks to buy


In [2]:
!pip install yahoo_fin
!pip install yfinance
!pip install yahoofinancials

In [3]:
import pandas as pd
import numpy as np

# import yahoo finance dependencies
import yahoo_fin.stock_info as si
import yfinance as yf
from yahoofinancials import YahooFinancials


# from plotly.offline import plot, init_notebook_mode
# init_notebook_mode()
# import cufflinks as cf
# cf.set_config_file(offline=True)

# sql dependencies
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float

import psycopg2
import psycopg2.extras as extras


In [4]:
dow = si.tickers_dow()
sp500 = si.tickers_sp500()
nasdaq = si.tickers_nasdaq()

In [5]:
dow

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CAT',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'V',
 'VZ',
 'WBA',
 'WMT']

In [6]:
# prices_sp500 = {}

# for ticker in sp500:
#   prices_sp500[ticker] = si.get_data(ticker)

In [7]:
aapl_df = yf.download('AAPL', 
                      start='2019-01-01', 
                      end='2021-06-12', 
                      progress=False,
                      
)
# aapl_df["ticker"]= np.repeat("AAPLE", aapl_df.shape[0])
aapl_df["ticker"] = "APPL"
aapl_df

,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2019-01-02,38.722500,39.712502,38.557499,39.480000,38.277519,148158800,APPL
2019-01-03,35.994999,36.430000,35.500000,35.547501,34.464802,365248800,APPL
2019-01-04,36.132500,37.137501,35.950001,37.064999,35.936081,234428400,APPL
2019-01-07,37.174999,37.207500,36.474998,36.982498,35.856094,219111200,APPL
2019-01-08,37.389999,37.955002,37.130001,37.687500,36.539623,164101200,APPL
...,...,...,...,...,...,...,...
2021-06-07,126.169998,126.320000,124.830002,125.900002,125.368721,71057600,APPL
2021-06-08,126.599998,128.460007,126.209999,126.739998,126.205177,74403800,APPL
2021-06-09,127.209999,127.750000,126.519997,127.129997,126.593529,56877900,APPL


In [13]:
dow_df = pd.DataFrame()
for ticker in dow:
    dow_dl = yf.download(ticker, start='2012-01-01', 
                      end='2022-04-01', progress=False)
    dow_dl['Ticker'] = ticker
    dow_dl['Market'] = "Dow"
#     dow_df['Ticker'] = np.repeat(ticker,dow_df.shape[0])

    dow_df = dow_df.append(dow_dl)

dow_df


,Open,High,Low,Close,Adj Close,Volume,Ticker,Market
Date,,,,,,,,
2012-01-03,14.621429,14.732143,14.607143,14.686786,12.575918,302220800,AAPL,Dow
2012-01-04,14.642857,14.810000,14.617143,14.765714,12.643501,260022000,AAPL,Dow
2012-01-05,14.819643,14.948214,14.738214,14.929643,12.783871,271269600,AAPL,Dow
2012-01-06,14.991786,15.098214,14.972143,15.085714,12.917510,318292800,AAPL,Dow
2012-01-09,15.196429,15.276786,15.048214,15.061786,12.897020,394024400,AAPL,Dow
...,...,...,...,...,...,...,...,...
2022-03-25,143.449997,143.929993,142.259995,143.449997,143.449997,4695100,WMT,Dow
2022-03-28,143.500000,146.059998,143.300003,146.000000,146.000000,6701900,WMT,Dow
2022-03-29,146.600006,148.100006,146.250000,147.229996,147.229996,6644800,WMT,Dow


In [14]:
sp_df = pd.DataFrame()
for ticker in sp500:
    sp_dl = yf.download(ticker, start='2012-01-01', 
                      end='2022-04-01', progress=False)
    sp_dl['Ticker'] = ticker
    sp_dl['Market'] = "SP500"
#     dow_df['Ticker'] = np.repeat(ticker,dow_df.shape[0])

    sp_df = sp_df.append(sp_dl)

sp_df



1 Failed download:
- CZR: No data found for this date range, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume,Ticker,Market
Date,,,,,,,,
2012-01-03,25.636623,26.459227,25.565092,26.094421,23.5227,4156394.0,A,SP500
2012-01-04,25.858370,26.008583,25.457796,25.886980,23.3357,4651845.0,A,SP500
2012-01-05,25.665236,26.695278,25.550787,26.466381,23.858,6842651.0,A,SP500
2012-01-06,26.530758,26.866953,26.330473,26.752504,24.116,4711400.0,A,SP500
2012-01-09,26.909870,27.482117,26.802574,27.453505,24.7479,4429563.0,A,SP500
...,...,...,...,...,...,...,...,...
2022-03-25,190.800003,191.580002,188.149994,189.429993,189.43,2363100.0,ZTS,SP500
2022-03-28,190.229996,191.419998,187.600006,189.369995,189.37,2995800.0,ZTS,SP500
2022-03-29,191.889999,194.600006,191.740005,192.279999,192.28,2557400.0,ZTS,SP500


In [15]:
frames = [dow_df, sp_df]
sp_dow_df = pd.concat(frames)
sp_dow_df

,Open,High,Low,Close,Adj Close,Volume,Ticker,Market
Date,,,,,,,,
2012-01-03,14.621429,14.732143,14.607143,14.686786,12.5759,302220800.0,AAPL,Dow
2012-01-04,14.642857,14.810000,14.617143,14.765714,12.6435,260022000.0,AAPL,Dow
2012-01-05,14.819643,14.948214,14.738214,14.929643,12.7839,271269600.0,AAPL,Dow
2012-01-06,14.991786,15.098214,14.972143,15.085714,12.9175,318292800.0,AAPL,Dow
2012-01-09,15.196429,15.276786,15.048214,15.061786,12.897,394024400.0,AAPL,Dow
...,...,...,...,...,...,...,...,...
2022-03-25,190.800003,191.580002,188.149994,189.429993,189.43,2363100.0,ZTS,SP500
2022-03-28,190.229996,191.419998,187.600006,189.369995,189.37,2995800.0,ZTS,SP500
2022-03-29,191.889999,194.600006,191.740005,192.279999,192.28,2557400.0,ZTS,SP500


In [17]:
sp_dow_df.to_csv('dataframes.csv', index =True)